In [1]:
import os
from pathlib import Path
# Change cwd to the project root (parent of 'notebooks/')
os.chdir(Path.cwd().parent)
Path.cwd()

PosixPath('/Users/jbrandt/code/birddog')

In [2]:
import json
import requests
from copy import copy, deepcopy
import mwparserfromhell

In [3]:
from birddog.wiki import (
    ARCHIVE_BASE, ARCHIVE_LIST, ARCHIVES, SUBARCHIVES, 
    update_master_archive_list
    )
from birddog.utility import translate_page, get_text, form_text_item, fetch_url 
from birddog.core import Archive
from birddog.translate import translation

2025-04-24 18:50:54,159 [INFO] Using Google Cloud translation API (credentials file:/Users/jbrandt/code/birddog/google-cloud-translate-key.json)
2025-04-24 18:50:54,197 [INFO] Found credentials in environment variables.
2025-04-24 18:50:54,240 [INFO] Using AWS S3 bucket birddog-data for storage.


In [4]:
ARCHIVE_LIST

{'DAARK': 'Архів:ДААРК',
 'DACHGO': 'Архів:ДАЧгО',
 'DACHKO': 'Архів:ДАЧкО',
 'DACHVO': 'Архів:ДАЧвО',
 'DADNO': 'Архів:ДАДнО',
 'DADO': 'Архів:ДАДоО',
 'DAHEO': 'Архів:ДАХеО',
 'DAHMO': 'Архів:ДАХмО',
 'DAHO': 'Архів:ДАХО',
 'DAIFO': 'Архів:ДАІФО',
 'DAK': 'Архів:ДАК',
 'DAKIRO': 'Архів:ДАКрО',
 'DAKO': 'Архів:ДАКО',
 'DAKRE': 'Архів:Архівний_відділ_виконавчого_комітету_Кременчуцької_міської_ради',
 'DALO': 'Архів:ДАЛО',
 'DALUO': 'Архів:ДАЛуО',
 'DAMO': 'Архів:ДАМО',
 'DAOO': 'Архів:ДАОО',
 'DAPO': 'Архів:ДАПО',
 'DARO': 'Архів:ДАРО',
 'DAS': 'Архів:ДАС',
 'DASO': 'Архів:ДАСО',
 'DATO': 'Архів:ДАТО',
 'DAVIO': 'Архів:ДАВіО',
 'DAVOO': 'Архів:ДАВоО',
 'DAZHO': 'Архів:ДАЖО',
 'DAZKO': 'Архів:ДАЗкО',
 'DAZPO': 'Архів:ДАЗпО',
 'DISZMO': 'Архів:ДІСЗМО'}

In [5]:
"""
these have no subarchive:
 "CDIAK":    "Архів:ЦДІАК",
 "CSDAG":    "Архів:ЦДАГО",
     "TSDAVO":   "Архів:ЦДАВО",
 "TSDIAL":   "Архів:ЦДІАЛ"    
     "GDA-MO":   "Архів:ГДА_МО",
    "GDA-MVS":  "Архів:ГДА_МВС",
    "GDA-SBU":  "Архів:ГДА_СБУ",
    "GDA-SZRU": "Архів:ГДА_СЗРУ",
        "IMFE":     "Архів:АНФРФ_ІМФЕ",
    "OMELNIK":  "Архів:Трудовий_архів_виконавчого_комітету_Омельницької_сільської_ради_Кременчуцького_району_Полтавської_області"


this one fails to find any subarchives:
    "DAKRE":    "Архів:Архівний_відділ_виконавчого_комітету_Кременчуцької_міської_ради",

"""

'\nthese have no subarchive:\n "CDIAK":    "Архів:ЦДІАК",\n "CSDAG":    "Архів:ЦДАГО",\n     "TSDAVO":   "Архів:ЦДАВО",\n "TSDIAL":   "Архів:ЦДІАЛ"    \n     "GDA-MO":   "Архів:ГДА_МО",\n    "GDA-MVS":  "Архів:ГДА_МВС",\n    "GDA-SBU":  "Архів:ГДА_СБУ",\n    "GDA-SZRU": "Архів:ГДА_СЗРУ",\n        "IMFE":     "Архів:АНФРФ_ІМФЕ",\n    "OMELNIK":  "Архів:Трудовий_архів_виконавчого_комітету_Омельницької_сільської_ради_Кременчуцького_району_Полтавської_області"\n\n\nthis one fails to find any subarchives:\n    "DAKRE":    "Архів:Архівний_відділ_виконавчого_комітету_Кременчуцької_міської_ради",\n\n'

In [6]:
#update_master_archive_list()

In [7]:
archive_master_list = [(arc, sub['subarchive']['en']) for arc, archive in ARCHIVES.items() for sub in archive.values()]
archive_master_list

[('DAARK', 'D'),
 ('DACHGO', 'D'),
 ('DACHGO', 'R'),
 ('DACHKO', 'D'),
 ('DACHKO', 'R'),
 ('DACHVO', 'D'),
 ('DACHVO', 'R'),
 ('DADNO', 'D'),
 ('DADNO', 'R'),
 ('DADO', 'D'),
 ('DAHEO', 'D'),
 ('DAHEO', 'R'),
 ('DAHEO', 'P'),
 ('DAHMO', 'D'),
 ('DAHMO', 'R'),
 ('DAHMO', 'P'),
 ('DAHMO', 'K'),
 ('DAHO', 'D'),
 ('DAHO', 'R'),
 ('DAIFO', 'D'),
 ('DAK', 'D'),
 ('DAK', 'R'),
 ('DAKIRO', 'D'),
 ('DAKIRO', 'R'),
 ('DAKIRO', 'P'),
 ('DAKO', 'D'),
 ('DAKO', 'R'),
 ('DAKO', 'P'),
 ('DAKRE', 'R'),
 ('DAKRE', 'OS'),
 ('DALO', 'D'),
 ('DALO', 'R'),
 ('DALO', 'P'),
 ('DALUO', 'D'),
 ('DALUO', 'R'),
 ('DAMO', 'D'),
 ('DAMO', 'R'),
 ('DAMO', 'P'),
 ('DAMO', 'A'),
 ('DAOO', 'D'),
 ('DAOO', 'R'),
 ('DAPO', 'D'),
 ('DAPO', 'R'),
 ('DAPO', 'P'),
 ('DARO', 'D'),
 ('DARO', 'R'),
 ('DAS', 'D'),
 ('DAS', 'P'),
 ('DASO', 'D'),
 ('DASO', 'R'),
 ('DATO', 'D'),
 ('DAVIO', 'D'),
 ('DAVIO', 'R'),
 ('DAVIO', 'P'),
 ('DAVOO', 'D'),
 ('DAVOO', 'R'),
 ('DAVOO', 'P'),
 ('DAZHO', 'D'),
 ('DAZHO', 'R'),
 ('DAZPO', 'D'),
 

In [8]:
SUBARCHIVES

[{'uk': 'Д', 'en': 'D'},
 {'uk': 'Р', 'en': 'R'},
 {'uk': 'П', 'en': 'P'},
 {'uk': 'К', 'en': 'K'},
 {'uk': 'ОС', 'en': 'OS'},
 {'uk': 'А', 'en': 'A'},
 {'uk': 'Цифрові колекції', 'en': 'Digital collections'}]

In [9]:
archive_master_list = [(arc, sub['subarchive']['en']) for arc, archive in ARCHIVES.items() for sub in archive.values()]
for item in archive_master_list:
    print(f'{item[0]}-{item[1]}')

DAARK-D
DACHGO-D
DACHGO-R
DACHKO-D
DACHKO-R
DACHVO-D
DACHVO-R
DADNO-D
DADNO-R
DADO-D
DAHEO-D
DAHEO-R
DAHEO-P
DAHMO-D
DAHMO-R
DAHMO-P
DAHMO-K
DAHO-D
DAHO-R
DAIFO-D
DAK-D
DAK-R
DAKIRO-D
DAKIRO-R
DAKIRO-P
DAKO-D
DAKO-R
DAKO-P
DAKRE-R
DAKRE-OS
DALO-D
DALO-R
DALO-P
DALUO-D
DALUO-R
DAMO-D
DAMO-R
DAMO-P
DAMO-A
DAOO-D
DAOO-R
DAPO-D
DAPO-R
DAPO-P
DARO-D
DARO-R
DAS-D
DAS-P
DASO-D
DASO-R
DATO-D
DAVIO-D
DAVIO-R
DAVIO-P
DAVOO-D
DAVOO-R
DAVOO-P
DAZHO-D
DAZHO-R
DAZPO-D
DAZPO-R
DISZMO-Digital collections


In [ ]:
def get_archive_links():
    # Define the API endpoint and parameters
    api_url = "https://uk.wikisource.org/w/api.php"
    params = {
        "action": "parse",
        "page": "Архів:Архіви",
        "prop": "wikitext",
        "format": "json"
    }

    # Send a GET request to the MediaWiki API
    data = fetch_url(api_url, params=params, json=True)
    wikitext = data['parse']['wikitext']['*']

    # Parse wikitext using mwparserfromhell
    wikicode = mwparserfromhell.parse(wikitext)

    # Extract archive links with descriptions
    archive_links = []
    for link in wikicode.filter_wikilinks():
        target = str(link.title).strip()
        if target.startswith("Архів:"):
            description = str(link.text).strip() if link.text else target
            archive_links.append({
                "link": form_text_item(target),
                "description": form_text_item(description)
            })

    return archive_links

In [ ]:
links = get_archive_links()
links

In [ ]:
trans_links = deepcopy(links)
translate_page(trans_links)
trans_links

In [ ]:
sorted([get_text(item["link"]).split(":")[1] for item in trans_links])

In [ ]:
def invert_dict(x):
    return {v: k for k, v in x.items()}

In [ ]:
invert_dict(ARCHIVE_LIST)

In [ ]:
def form_archive_list():
    links = get_archive_links()
    trans_links = deepcopy(links)
    translate_page(trans_links)
    return {get_text(item["link"]).split(":")[1]:item["link"]["uk"] for item in trans_links}

In [ ]:
archive_list = form_archive_list()

In [ ]:
archive_list

In [ ]:
inv_archive_list = invert_dict(archive_list)
inv_archive_list

In [ ]:
update_mapping = { key: inv_archive_list[value] for key, value in ARCHIVE_LIST.items()}
update_mapping